In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

df = pd.read_json("https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard/raw/main/gorilla_openfunctions_v1_test_simple.json", lines=True)
df.head()

,question,function
0,Find the area of a triangle with a base of 10 ...,"{'name': 'calculate_triangle_area', 'descripti..."
1,Calculate the factorial of 5 using math functi...,"{'name': 'math.factorial', 'description': 'Cal..."
2,Calculate the hypotenuse of a right triangle g...,"{'name': 'math.hypot', 'description': 'Calcula..."
3,Find the roots of a quadratic equation with co...,"{'name': 'algebra.quadratic_roots', 'descripti..."
4,"Solve a quadratic equation where a=2, b=6, and...","{'name': 'solve_quadratic_equation', 'descript..."


In [3]:
from llama_cpp import Llama

llm = Llama("/home/asilva/models/Mistral-7B-Instruct-v0.3.fp16.gguf", n_gpu_layers=-1)

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from /home/asilva/models/Mistral-7B-Instruct-v0.3.fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mistral-7B-Instruc...
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv

In [4]:
pos = 0
question = df.iloc[pos]["question"]
question

'Find the area of a triangle with a base of 10 units and height of 5 units.'

In [5]:
import json

function = df.iloc[pos]['function']
json.dumps(function)

'{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}'

In [6]:
final_prompt = (
    "<s>[AVAILABLE_TOOLS] ["
    + json.dumps(function)
    + "] [/AVAILABLE_TOOLS][INST] "
    + question
    + " [/INST]"
    + "[TOOL_CALLS] "
)
final_prompt

'<s>[AVAILABLE_TOOLS] [{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}] [/AVAILABLE_TOOLS][INST] Find the area of a triangle with a base of 10 units and height of 5 units. [/INST][TOOL_CALLS] '

In [7]:
response = llm(final_prompt, temperature=0, seed=0, max_tokens=None)

/home/asilva/test-finite-state-machine/.venv/lib/python3.10/site-packages/llama_cpp/llama.py:1054: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(

llama_print_timings:        load time =     196.58 ms
llama_print_timings:      sample time =      17.11 ms /    45 runs   (    0.38 ms per token,  2630.19 tokens per second)
llama_print_timings: prompt eval time =     196.15 ms /   154 tokens (    1.27 ms per token,   785.13 tokens per second)
llama_print_timings:        eval time =    1724.01 ms /    44 runs   (   39.18 ms per token,    25.52 tokens per second)
llama_print_timings:       total time =    1949.86 ms /   198 tokens


In [8]:
response["choices"][0]["text"]

' [{"name": "calculate_triangle_area", "arguments": {"base": 10, "height": 5}}]\n\nThe area of the triangle is 50 square units.'

In [9]:
import regex

def extract_first_of_nested_brackets(input_string):
    # pattern = r'\{(?>[^{}]+|(?R))*\}'
    pattern = r'\[(.*?)\]'
    match = regex.search(pattern, input_string)
    if match:
        return match.group(0)[1:-1]
    return None

In [10]:
final_result = extract_first_of_nested_brackets(response["choices"][0]["text"])
final_result

'{"name": "calculate_triangle_area", "arguments": {"base": 10, "height": 5}}'

In [11]:
import json_repair
json_repair.loads(final_result)

{'name': 'calculate_triangle_area', 'arguments': {'base': 10, 'height': 5}}

In [12]:
GORILLA_TO_OPENAPI = {
    "integer": "integer",
    "number": "number",
    "float": "number",
    "string": "string",
    "boolean": "boolean",
    "bool": "boolean",
    "array": "array",
    "list": "array",
    "dict": "object",
    "object": "object",
    "tuple": "array",
    "any": "string",
    "byte": "integer",
    "short": "integer",
    "long": "integer",
    "double": "number",
    "char": "string",
    "ArrayList": "array",
    "Array": "array",
    "HashMap": "object",
    "Hashtable": "object",
    "Queue": "array",
    "Stack": "array",
    "Any": "string",
    "String": "string",
    "Bigint": "integer",
}

In [13]:
def _cast_to_openai_type(properties, mapping, test_category):
    for key, value in properties.items():
        if "type" not in value:
            properties[key]["type"] = "string"
        else:
            var_type = value["type"]
            if mapping == GORILLA_TO_OPENAPI and var_type == "float":
                properties[key]["format"] = "float"
                properties[key]["description"] += " This is a float type value."
            if var_type in mapping:
                properties[key]["type"] = mapping[var_type]
            else:
                properties[key]["type"] = "string"

        # Currently support:
        # - list of any
        # - list of list of any
        # - list of dict
        # - list of list of dict
        # - dict of any

        if properties[key]["type"] == "array" or properties[key]["type"] == "object":
            if "properties" in properties[key]:
                properties[key]["properties"] = _cast_to_openai_type(
                    properties[key]["properties"], mapping, test_category
                )
            elif "items" in properties[key]:
                properties[key]["items"]["type"] = mapping[
                    properties[key]["items"]["type"]
                ]
                if (
                    properties[key]["items"]["type"] == "array"
                    and "items" in properties[key]["items"]
                ):
                    properties[key]["items"]["items"]["type"] = mapping[
                        properties[key]["items"]["items"]["type"]
                    ]
                elif (
                    properties[key]["items"]["type"] == "object"
                    and "properties" in properties[key]["items"]
                ):
                    properties[key]["items"]["properties"] = _cast_to_openai_type(
                        properties[key]["items"]["properties"], mapping, test_category
                    )
    return properties

In [14]:
function["parameters"]["properties"]

{'base': {'type': 'integer', 'description': 'The base of the triangle.'},
 'height': {'type': 'integer', 'description': 'The height of the triangle.'},
 'unit': {'type': 'string',
  'description': "The unit of measure (defaults to 'units' if not specified)"}}

In [15]:
_cast_to_openai_type(function["parameters"]["properties"], GORILLA_TO_OPENAPI, "simple")

{'base': {'type': 'integer', 'description': 'The base of the triangle.'},
 'height': {'type': 'integer', 'description': 'The height of the triangle.'},
 'unit': {'type': 'string',
  'description': "The unit of measure (defaults to 'units' if not specified)"}}

In [16]:
# allow `\"`, `\\`, or any character which isn't a control sequence
STRING_INNER = r'([^"\\\x00-\x1F\x7F-\x9F]|\\["\\])'
STRING = f'"{STRING_INNER}*"'

INTEGER = r"(-)?(0|[1-9][0-9]*)"
NUMBER = rf"({INTEGER})(\.[0-9]+)?([eE][+-][0-9]+)?"
BOOLEAN = r"(true|false)"
NULL = r"null"
WHITESPACE = r"[ ]?"

type_to_regex = {
    "string": STRING,
    "integer": INTEGER,
    "number": NUMBER,
    "boolean": BOOLEAN,
    "null": NULL,
}

In [17]:
function_data = function

In [18]:
for arg, value in _cast_to_openai_type(
            function_data['parameters']['properties'], 
            GORILLA_TO_OPENAPI, 
            "simple").items():
        # do any of the examples use the option parameter?
        # Easy enough to add in!
        if arg in function_data['parameters']['required']:
            print(arg)
            print(value)

base
{'type': 'integer', 'description': 'The base of the triangle.'}
height
{'type': 'integer', 'description': 'The height of the triangle.'}


In [19]:
# this should be more universal
def type_to_regex(arg_meta):
    basic_map = {
        "string": f'"{STRING_INNER}{{1,42}}"', # might need to be longer?
        "integer": INTEGER,
        "number": NUMBER,
        "float": NUMBER, # change this later
        "boolean": BOOLEAN,
        "null": NULL,
    }
    arg_type = arg_meta['type']
    if arg_type == 'object':
        arg_type = 'dict'
    if arg_type == 'dict':
        result = build_dict_regex(arg_meta['properties'])
    # Note, this currently won't pass the empty list
    elif arg_type == 'array':
        pattern = type_to_regex(arg_meta['items'])
        result = array_regex = r'\[('+pattern+', ){0,8}'+pattern+'\]'
    else:
        result = basic_map[arg_type]
    return result


In [20]:
for arg, value in _cast_to_openai_type(
    function_data['parameters']['properties'], 
    GORILLA_TO_OPENAPI, 
    "simple").items():
# do any of the examples use the option parameter?
# Easy enough to add in!
    if arg in function_data['parameters']['required']:
        print(arg)
        print(value)
        print(type_to_regex(value))

base
{'type': 'integer', 'description': 'The base of the triangle.'}
(-)?(0|[1-9][0-9]*)
height
{'type': 'integer', 'description': 'The height of the triangle.'}
(-)?(0|[1-9][0-9]*)


In [21]:
def build_fc_regex(function_data):
    out_re = r'\[\{"name": "' + function_data["name"] + '", "arguments": \{'        
    args_part  = ", ".join([
        f'"{arg}": '+type_to_regex(value)
        for arg, value in _cast_to_openai_type(
            function_data['parameters']['properties'], 
            GORILLA_TO_OPENAPI, 
            "simple").items()
        # do any of the examples use the option parameter?
        # Easy enough to add in!
        if arg in function_data['parameters']['required']
    ])
    optional_part = "".join([
        f'(, "{arg}": '+type_to_regex(value) + r')?'
        for arg, value in _cast_to_openai_type(
            function_data['parameters']['properties'], 
            GORILLA_TO_OPENAPI, 
            "simple").items()
        if not (arg in function_data['parameters']['required'])
    ])
    return out_re+args_part+optional_part+"}}]"

In [22]:
function

{'name': 'calculate_triangle_area',
 'description': 'Calculate the area of a triangle given its base and height.',
 'parameters': {'type': 'dict',
  'properties': {'base': {'type': 'integer',
    'description': 'The base of the triangle.'},
   'height': {'type': 'integer', 'description': 'The height of the triangle.'},
   'unit': {'type': 'string',
    'description': "The unit of measure (defaults to 'units' if not specified)"}},
  'required': ['base', 'height']}}

In [23]:
regex = build_fc_regex(function)
regex

'\\[\\{"name": "calculate_triangle_area", "arguments": \\{"base": (-)?(0|[1-9][0-9]*), "height": (-)?(0|[1-9][0-9]*)(, "unit": "([^"\\\\\\x00-\\x1F\\x7F-\\x9F]|\\\\["\\\\]){1,42}")?}}]'

In [24]:
from outlines import models

model = models.LlamaCpp(llm)

/home/asilva/test-finite-state-machine/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
from outlines import generate

generator = generate.regex(model, regex)

In [26]:
final_prompt

'<s>[AVAILABLE_TOOLS] [{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}] [/AVAILABLE_TOOLS][INST] Find the area of a triangle with a base of 10 units and height of 5 units. [/INST][TOOL_CALLS] '

In [27]:
answer = generator(final_prompt, max_tokens=1024)

/home/asilva/test-finite-state-machine/.venv/lib/python3.10/site-packages/llama_cpp/llama.py:1054: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
Llama.generate: prefix-match hit

llama_print_timings:        load time =     196.58 ms
llama_print_timings:      sample time =      36.02 ms /    31 runs   (    1.16 ms per token,   860.70 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    1238.32 ms /    31 runs   (   39.95 ms per token,    25.03 tokens per second)
llama_print_timings:       total time =    1285.84 ms /    31 tokens


In [28]:
answer

'[{"name": "calculate_triangle_area", "arguments": {"base": 10, "height": 5}}]'

In [29]:
answers = []
for pos in range(len(df)):
    try:
        print(pos)
        question = df.iloc[pos]["question"]
        function = df.iloc[pos]['function']
        final_prompt = (
            "<s>[AVAILABLE_TOOLS] ["
            + json.dumps(function)
            + "] [/AVAILABLE_TOOLS][INST] "
            + question
            + " [/INST]"
            + "[TOOL_CALLS] "
        )
        regex = build_fc_regex(function)
        generator = generate.regex(model, regex)
        answer = generator(final_prompt, max_tokens=1024)
        answers.append(answer[1:-1])
    except:
        answers.append("Error")

0



llama_print_timings:        load time =     196.58 ms
llama_print_timings:      sample time =      36.27 ms /    31 runs   (    1.17 ms per token,   854.80 tokens per second)
llama_print_timings: prompt eval time =      74.37 ms /   154 tokens (    0.48 ms per token,  2070.84 tokens per second)
llama_print_timings:        eval time =    1181.19 ms /    30 runs   (   39.37 ms per token,    25.40 tokens per second)
llama_print_timings:       total time =    1301.41 ms /   184 tokens


1



llama_print_timings:        load time =     196.58 ms
llama_print_timings:      sample time =      24.32 ms /    21 runs   (    1.16 ms per token,   863.63 tokens per second)
llama_print_timings: prompt eval time =      59.00 ms /    93 tokens (    0.63 ms per token,  1576.22 tokens per second)
llama_print_timings:        eval time =     783.28 ms /    20 runs   (   39.16 ms per token,    25.53 tokens per second)
llama_print_timings:       total time =     874.21 ms /   113 tokens


2


...


llama_print_timings:        load time =     196.58 ms
llama_print_timings:      sample time =      49.24 ms /    42 runs   (    1.17 ms per token,   852.93 tokens per second)
llama_print_timings: prompt eval time =      75.84 ms /   159 tokens (    0.48 ms per token,  2096.55 tokens per second)
llama_print_timings:        eval time =    1614.48 ms /    41 runs   (   39.38 ms per token,    25.40 tokens per second)
llama_print_timings:       total time =    1763.91 ms /   200 tokens


384


Compiling FSM index for all state transitions:  41%|████▏     | 477/1153 [00:06<00:08, 79.29it/s] 

llama_print_timings:        load time =     196.58 ms
llama_print_timings:      sample time =      89.83 ms /    73 runs   (    1.23 ms per token,   812.66 tokens per second)
llama_print_timings: prompt eval time =     131.66 ms /   305 tokens (    0.43 ms per token,  2316.61 tokens per second)
llama_print_timings:        eval time =    2864.01 ms /    72 runs   (   39.78 ms per token,    25.14 tokens per second)
llama_print_timings:       total time =    3159.50 ms /   377 tokens


385


Compiling FSM index for all state transitions:  22%|██▏       | 199/909 [00:02<00:11, 63.84it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [31]:
answers[-5:]

['{"name": "parking_lot.find_nearest", "arguments": {"location": "Central Park, NY", "radius": 2}}',
 '{"name": "hospital.locate", "arguments": {"location": "Denver, CO", "radius": 5, "department": "Pediatrics"}}',
 '{"name": "distance_calculator.calculate", "arguments": {"origin": "New York", "destination": "Boston", "consider_terrain": true}}',
 '{"name": "get_museum_hours", "arguments": {"museum_name": "Metropolitan Museum of Art", "day": "Saturday"}}',
 '{"name": "restaurant_search", "arguments": {"location": "New York City, NY", "cuisine": "Italian", "rating": 4, "accepts_credit_cards": true}}']

In [32]:
len(answers)

400

In [33]:
# RUN IT ONCE TO DOWNLOAD THE TEST DATASET
# !wget https://raw.githubusercontent.com/ShishirPatil/gorilla/main/berkeley-function-call-leaderboard/data/possible_answer/gorilla_openfunctions_v1_test_simple.json

In [34]:
file_path = "gorilla_openfunctions_v1_test_simple.json"

In [35]:
# standardize possible_answers from gorilla simple test set
import json

possible_answers = []
# Open the file and read line by line
with open(file_path, 'r') as file:
    for line in file:
        # Parse the JSON line into a Python dictionary
        json_data = json.loads(line)
        
        # Process the JSON object
        for function_name, params in json_data.items():
            final_string = function_name + ',{'
            for param_name, param_value in params.items():
                final_string += '"' + param_name + '":' + str(param_value) +','
            final_string = final_string[:-1] +"}"
            final_string = final_string.replace("'", '"')
            final_string = final_string.replace(" ", "")
            possible_answers.append(final_string)

In [36]:
answers[0]

'{"name": "calculate_triangle_area", "arguments": {"base": 10, "height": 5}}'

In [37]:
# standardize model_output
def standardize(result):
    final_result = str(json_repair.loads(result)["name"]) + "," + str(json_repair.loads(result)["arguments"])
    final_result = final_result.replace("'",'"')
    final_result = final_result.replace(" ", "")
    return final_result

In [38]:
model_output = []
for i, result in enumerate(answers):
    try:
        model_output.append(standardize(result))
    except:
        model_output.append("Error")

In [39]:
model_output[:5]

['calculate_triangle_area,{"base":10,"height":5}',
 'math.factorial,{"number":5}',
 'math.hypot,{"x":4,"y":5}',
 'algebra.quadratic_roots,{"a":1,"b":-3,"c":2}',
 'solve_quadratic_equation,{"a":2,"b":6,"c":5}']

In [40]:
df = pd.DataFrame()
df["possible_answer"] = possible_answers
df["model_output"] = model_output
df.head()

,possible_answer,model_output
0,"calculate_triangle_area,{""base"":[10],""height"":...","calculate_triangle_area,{""base"":10,""height"":5}"
1,"math.factorial,{""number"":[5]}","math.factorial,{""number"":5}"
2,"math.hypot,{""x"":[4],""y"":[5],""z"":["""",0]}","math.hypot,{""x"":4,""y"":5}"
3,"algebra.quadratic_roots,{""a"":[1],""b"":[-3],""c"":...","algebra.quadratic_roots,{""a"":1,""b"":-3,""c"":2}"
4,"solve_quadratic_equation,{""a"":[2],""b"":[6],""c"":...","solve_quadratic_equation,{""a"":2,""b"":6,""c"":5}"


In [41]:
df.to_csv("results_Outlines_Mistral_v0_3.csv", index=False)